In [10]:
import io
import json
import psycopg2
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
def write_to_table(df,schema,table_name,password='1q2w3e4r',host='127.0.0.1',DB='RENT',if_exists='append',port='5432'):
    db_engine = create_engine('postgresql://postgres:{}@{}:{}/{}'.format(password,host,port,DB))
    string_data_io = io.StringIO()
    df.to_csv(string_data_io,sep=',',index=False)
    pd_sql_engine = pd.io.sql.pandasSQL_builder(db_engine)
    table = pd.io.sql.SQLTable(table_name,pd_sql_engine,frame=df,index=False,if_exists=if_exists,schema=schema)
    table.create()
    
    string_data_io.seek(0)
    string_data_io.readline()
    with db_engine.connect() as connection:
        with connection.connection.cursor() as cursor:
            copy_cmd = '''COPY "%s"."%s" FROM STDIN HEADER DELIMITER ',' CSV''' % (schema, table_name)
            cursor.copy_expert(copy_cmd, string_data_io)
        connection.connection.commit()

In [3]:
conn = psycopg2.connect(database='Community',user='postgres',password='1q2w3e4r',host='127.0.0.1',port=5432)

In [4]:
with open("bk_city_map.json", 'r', encoding='utf-8') as fp:
    city_dist = json.loads(fp.read())
city_list = []
for city, districts in city_dist.items():
    city_list.append(city)

In [5]:
city_list

['合肥',
 '芜湖',
 '马鞍山',
 '安庆',
 '北京',
 '重庆',
 '福州',
 '厦门',
 '泉州',
 '漳州',
 '广州',
 '深圳',
 '珠海',
 '佛山',
 '江门',
 '湛江',
 '惠州',
 '东莞',
 '中山',
 '清远',
 '南宁',
 '柳州',
 '桂林',
 '北海',
 '防城港',
 '贵阳',
 '遵义',
 '兰州',
 '石家庄',
 '唐山',
 '邯郸',
 '保定',
 '廊坊',
 '张家口',
 '哈尔滨',
 '郑州',
 '开封',
 '洛阳',
 '新乡',
 '许昌',
 '信阳',
 '周口',
 '驻马店',
 '武汉',
 '黄石',
 '宜昌',
 '襄阳',
 '鄂州',
 '孝感',
 '黄冈',
 '长沙',
 '株洲',
 '衡阳',
 '岳阳',
 '常德',
 '永州',
 '海口',
 '三亚',
 '澄迈',
 '长春',
 '吉林',
 '南京',
 '无锡',
 '徐州',
 '常州',
 '苏州',
 '昆山',
 '南通',
 '连云港',
 '淮安',
 '盐城',
 '镇江',
 '海门',
 '江阴',
 '南昌',
 '景德镇',
 '萍乡',
 '九江',
 '新余',
 '赣州',
 '吉安',
 '宜春',
 '抚州',
 '上饶',
 '沈阳',
 '大连',
 '丹东',
 '呼和浩特',
 '包头',
 '赤峰',
 '银川',
 '太原',
 '大同',
 '运城',
 '晋中',
 '济南',
 '青岛',
 '淄博',
 '烟台',
 '潍坊',
 '济宁',
 '泰安',
 '威海',
 '临沂',
 '菏泽',
 '上海',
 '成都',
 '泸州',
 '绵阳',
 '遂宁',
 '内江',
 '南充',
 '眉山',
 '宜宾',
 '达州',
 '巴中',
 '凉山',
 '乐山',
 '广元',
 '西安',
 '宝鸡',
 '咸阳',
 '汉中',
 '天津',
 '乌鲁木齐',
 '昆明',
 '大理',
 '杭州',
 '宁波',
 '温州',
 '嘉兴',
 '湖州',
 '绍兴',
 '金华',
 '义乌',
 '台州']

In [15]:
for city in city_list:
    sql = f"""SELECT id, "城市", "区县", "小区", "类型", "单价", "在售套数", "在租套数", "建筑年份", "longitude", "latitude", "涨跌幅", "建筑面积", "占地面积", "房屋总数", "楼栋总数","绿化率","容积率","产权描述","地址","小区url","抓取年份","抓取月份","数据来源"
        FROM public."Community_2020" where 城市='{city}';"""
    df_id = pd.read_sql_query(sql, con=conn)
    df_id.replace(np.nan, 0, inplace=True)
    df_id.replace(np.inf, 0, inplace=True)
    df_id['建筑年份'] = df_id['建筑年份'].astype(int)
    df_id['在租套数'] = df_id['在租套数'].astype(int)
    df_id['房屋总数'] = df_id['房屋总数'].astype(int)
    df_id['楼栋总数'] = df_id['楼栋总数'].astype(int)
    # print(df_id.shape)
    # df_id.drop_duplicates(['标题', '标题url'], inplace=True)
    print(df_id.shape)
    write_to_table(df_id, schema='public', table_name='Community_2020',password='1q2w3e4r', host='192.168.88.254',DB='Community' ,port='15432')
    

(4367, 24)
(1121, 24)
(828, 24)
(1297, 24)
(10366, 24)
(9168, 24)
(2994, 24)
(2539, 24)
(2046, 24)
(939, 24)
(7078, 24)
(4844, 24)
(1804, 24)
(5043, 24)
(1450, 24)
(1881, 24)
(1817, 24)
(1750, 24)
(2536, 24)
(0, 24)
(2156, 24)
(2047, 24)
(1922, 24)
(1370, 24)
(0, 24)
(3861, 24)
(0, 24)
(3296, 24)
(4394, 24)
(1745, 24)
(0, 24)
(1716, 24)
(1615, 24)
(2049, 24)
(3024, 24)
(7139, 24)
(1198, 24)
(2461, 24)
(1501, 24)
(1502, 24)
(0, 24)
(0, 24)
(0, 24)
(8020, 24)
(1473, 24)
(0, 24)
(1965, 24)
(0, 24)
(385, 24)
(0, 24)
(3963, 24)
(1581, 24)
(0, 24)
(1607, 24)
(1099, 24)
(0, 24)
(2095, 24)
(0, 24)
(0, 24)
(3331, 24)
(1328, 24)
(5384, 24)
(2935, 24)
(2690, 24)
(1839, 24)
(5478, 24)
(0, 24)
(2669, 24)
(0, 24)
(1364, 24)
(1803, 24)
(1420, 24)
(0, 24)
(0, 24)
(3335, 24)
(0, 24)
(325, 24)
(1648, 24)
(0, 24)
(2100, 24)
(2040, 24)
(0, 24)
(271, 24)
(1499, 24)
(3889, 24)
(3453, 24)
(729, 24)
(2936, 24)
(0, 24)
(926, 24)
(1749, 24)
(3528, 24)
(0, 24)
(0, 24)
(0, 24)
(4294, 24)
(5689, 24)
(2064, 24)
(27